In [1]:
# 此網頁以Asp.net寫成 比較特殊，可以以Selenium抓取

In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [220]:
import time

# 開始抓取

In [427]:
Big_List = []

url = "http://apis.afa.gov.tw/pagepub/AppContentPage.aspx?itemNo=PRI075"
driver = webdriver.Chrome("/Users/deniel3678/Desktop/Python/爬蟲/chromedriver")
driver.get(url)


element = driver.find_element_by_id("PRI095")  #各地價格查詢
element.click()

time.sleep(9)

driver.find_element_by_id("WR1_2_Q_AvgPriceType_C1_1").click() #月平均價格



for i in range(1,13,1):
    time.sleep(7)
    Select(driver.find_element_by_id("WR1_2_Q_PRSR_Month_C1")).select_by_visible_text("{}".format(i))     # 第幾個月
    Select(driver.find_element_by_id("WR1_2_Q_GroupCode_XX_C1")).select_by_visible_text(u"蔬菜類")


    for i in range(78):
        time.sleep(3)
        driver.find_element_by_css_selector("option[value=\"01\"]").click()
        driver.find_element_by_id("WR1_2_PRMG_01_{}".format(i)).click()                          # crop 0~77
        driver.find_element_by_css_selector("a.CSS_ABS_NormalLink > span").click()

        # switch to new window
        time.sleep(5)
        window_after = driver.window_handles[1]   # [1]代表新開啟的視窗位置
        driver.switch_to_window(window_after)
        soup = BeautifulSoup(driver.page_source,"lxml")



        p = []
        l = []
        n = []
        try:
            for i in range(4,50,2):
                a = soup.find_all("td")[i].string
                b = soup.find_all("td")[i+1].string
                p.append(a)
                l.append(b)
        except:
            pass
        n.append(soup.find_all("span")[2].string)
        n = n*len(p)
        df = pd.DataFrame({'農會': p,'價格': l, '作物': n})


        Big_List.append(df)   # 每月每項目為一個DataFrame。


        driver.close()

        window_before = driver.window_handles[0]
        driver.switch_to_window(window_before)

In [462]:
veg = pd.concat(Big_List,axis=0)  # 把不同DataFrame串起來

In [463]:
veg.reset_index(inplace=True)
del veg["index"]

In [467]:
veg.head()

,作物,價格,農會
0,作物名稱：蘿蔔,6.17,雲林縣_四湖鄉農會 _01月
1,作物名稱：蘿蔔,6.67,彰化縣_芳苑鄉農會 _01月
2,作物名稱：蘿蔔,6.17,雲林縣_台西鄉農會 _01月
3,作物名稱：蘿蔔,7,嘉義縣_布袋鎮農會 _01月
4,作物名稱：蘿蔔,6.21,彰化縣_福興鄉農會 _01月
